In [13]:
import numpy as np
import pandas as pd
import os

In [14]:
registered_business_path = os.path.join('Cleaned Datasets', 'cleaned_registered_businesses.csv')
health_scores_path = os.path.join('Cleaned Datasets', 'health_scores.csv')

In [15]:
registered_business_df = pd.read_csv(registered_business_path)
health_scores_df = pd.read_csv(health_scores_path)

In [25]:
registered_business_df.head(50)

,location_id,ownership_name,business_name,category,lic_code,street_address,State,City,zipcode,business_start_date,business_end_date,neighborhoods,business_location,supervisor_district,Lifespan,closed,age_if_opened
0,0376718-03-001,Chan Renee W,Blossoming Lotus Acupuncture,Private Education and Health Services,NaN,2670 Leavenworth St A,CA,San Francisco,94109.0,01/28/2010,NaN,North Beach,"2670 LEAVENWORTH ST\nA San Francisco, CA 94109\n",2.0,NaN,no,3104.0
1,1045798-12-151,Benhammou Ahmida,Zaalouk Market & Deli Grocery,Retail Trade,NaN,1209 Sutter St,CA,San Francisco,94109.0,12/01/2015,NaN,Nob Hill,"1209 SUTTER ST\nSAN FRANCISCO, CA 94109\n(37.7...",3.0,NaN,no,971.0
2,1170655-12-171,San Francisco Security Services Llc,San Francisco Security Services Llc,NaN,NaN,588 Sutter St,CA,San Francisco,94102.0,12/01/2017,NaN,NaN,NaN,NaN,NaN,no,240.0
3,0439260-05-001,Tyrala Glenn D,San Francisco Victorian Door,NaN,NaN,327 Highland Ave,CA,San Francisco,94110.0,05/27/2009,NaN,Bernal Heights,"327 HIGHLAND AVE\nSan Francisco, CA 94110\n(37...",9.0,NaN,no,3350.0
4,1172877-12-171,Three R Properties,Three R Properties,Accommodations,NaN,251 Diamond Street St Fl,CA,San Francisco,94114.0,12/27/2017,NaN,Castro/Upper Market,NaN,8.0,NaN,no,214.0
5,1172879-12-171,"Rhi-Mosser 275 Turk, Lp","Rhi-Mosser 275 Turk, Lp",Real Estate and Rental and Leasing Services,NaN,275 Turk St,CA,San Francisco,94102.0,11/28/2017,NaN,NaN,NaN,NaN,NaN,no,243.0
6,0920982-02-001,Pitocchelli Johanna,Marina Assist,Certain Services,NaN,2010 Chestnut St 308,CA,San Francisco,94123.0,06/15/1993,NaN,Marina,"2010 CHESTNUT ST\nSan Francisco, CA 94123\n(37...",2.0,NaN,no,9175.0
7,1173616-12-171,Ariel Sultan,Ariel Sultan,Accommodations,NaN,180 Carl St Apt 1,CA,San Francisco,94117.0,12/27/2017,NaN,Haight Ashbury,NaN,5.0,NaN,no,214.0
8,1173736-12-171,Claire Ogden,1852 Church Street Rental,Accommodations,NaN,1852 Church St,CA,San Francisco,94131.0,01/01/2017,05/24/2018,Glen Park,NaN,8.0,508.0,yes,NaN
9,0341035-02-001,Van Der Zee Peter & Kathryn,1-11 Lilac St Apts,Real Estate and Rental and Leasing Services,NaN,1-11 Lilac St,CA,San Francisco,94110.0,07/31/1996,NaN,Mission,"1 11 LILAC ST\nSan Francisco, CA 94110\n(37.75...",9.0,NaN,no,8033.0


In [26]:
health_scores_df['address'] = health_scores_df['address'].str.lower()
health_scores_df['name'] = health_scores_df['name'].str.lower()
health_scores_df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number,score,date,type
0,19,nrgize lifestyle cafe,"1200 van ness ave, 3rd floor",San Francisco,CA,94109,37.786848,-122.421547,14157763262,94.0,20171211.0,routine
1,19,nrgize lifestyle cafe,"1200 van ness ave, 3rd floor",San Francisco,CA,94109,37.786848,-122.421547,14157763262,96.0,20180607.0,routine
2,19,nrgize lifestyle cafe,"1200 van ness ave, 3rd floor",San Francisco,CA,94109,37.786848,-122.421547,14157763262,94.0,20160513.0,routine
3,24,omni s.f. hotel - 2nd floor pantry,"500 california st, 2nd floor",San Francisco,CA,94104,37.792888,-122.403135,14156779494,96.0,20160311.0,routine
4,24,omni s.f. hotel - 2nd floor pantry,"500 california st, 2nd floor",San Francisco,CA,94104,37.792888,-122.403135,14156779494,98.0,20161005.0,routine


In [27]:
merged_df = registered_business_df.merge(health_scores_df, left_on="business_name", right_on="name")

In [28]:
len(merged_df)

9